In [1]:
import ast
import os
os.chdir(os.pardir)

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from src import load_point_cloud
from src import rms_angle_error

from estimator import NormalEstimator

In [2]:
data_path = 'data'

In [3]:
with open(os.path.join(data_path, 'testset_no_noise.txt')) as f:
    testset_no_noise = [line.strip() for line in f]
with open(os.path.join(data_path, 'testset_low_noise.txt')) as f:
    testset_low_noise = [line.strip() for line in f]
with open(os.path.join(data_path, 'testset_med_noise.txt')) as f:
    testset_med_noise = [line.strip() for line in f]
with open(os.path.join(data_path, 'testset_high_noise.txt')) as f:
    testset_high_noise = [line.strip() for line in f]
with open(os.path.join(data_path, 'testset_vardensity_gradient.txt')) as f:
    testset_vardensity_gradient = [line.strip() for line in f]
with open(os.path.join(data_path, 'testset_vardensity_striped.txt')) as f:
    testset_vardensity_striped = [line.strip() for line in f]

In [4]:
class_dispatcher = {'no_noise': testset_no_noise,
                    'low_noise': testset_low_noise,
                    'med_noise': testset_med_noise,
                    'high_noise': testset_high_noise,
                    'gradient': testset_vardensity_gradient,
                    'striped': testset_vardensity_striped}

In [5]:
df_param_opt = pd.read_csv(os.path.join('estimator', 'param_opt.csv'),
                           index_col=0)
df_param_opt['params'] = df_param_opt['params'].apply(
    lambda row: ast.literal_eval(row)
)
df_param_opt

,class,params,mean_test_score
138,gradient,"{'gamma': 1, 'k': 30, 'kernel': 'rbf', 's': 0}",-6.450299
103,high_noise,"{'gamma': 1, 'k': 100, 'kernel': 'inverse_mult...",-34.935803
104,low_noise,"{'gamma': 1, 'k': 100, 'kernel': 'inverse_mult...",-11.715842
255,med_noise,"{'gamma': 3, 'k': 100, 'kernel': 'rbf', 's': N...",-25.449993
130,no_noise,"{'gamma': 1, 'k': 30, 'kernel': 'gaussian', 's...",-6.485461
131,striped,"{'gamma': 1, 'k': 30, 'kernel': 'gaussian', 's...",-6.414900


In [6]:
def run_test(cls):
    res = {'class': [],
           'dataset': [],
           'params': [],
           'score': []}
    for dataset in tqdm(class_dispatcher[cls]):
        xyz, n = load_point_cloud(dataset)
        params = df_param_opt.loc[
            df_param_opt['class'] == cls, 'params'
        ].item()
        estimator = NormalEstimator(**params).fit(xyz)
        n_estim = estimator.predict(xyz)
        angle_err = rms_angle_error(n_estim, n, orient=False)
        res['class'].append(cls)
        res['dataset'].append(dataset)
        res['params'].append(str(params))
        res['score'].append(angle_err)
    return res

In [7]:
res_no_noise = run_test('no_noise')

  0%|          | 0/19 [00:00<?, ?it/s]

In [8]:
res_med_noise = run_test('med_noise')

  0%|          | 0/19 [00:00<?, ?it/s]

In [10]:
res_low_noise = run_test('low_noise')

  0%|          | 0/19 [00:00<?, ?it/s]

In [11]:
res_high_noise = run_test('high_noise')

  0%|          | 0/19 [00:00<?, ?it/s]

In [12]:
res_gradient = run_test('gradient')

  0%|          | 0/16 [00:00<?, ?it/s]

/home/alk/miniconda3/envs/3d/lib/python3.9/site-packages/scipy/interpolate/_fitpack2.py:1419: UserWarning: 
The required storage space exceeds the available storage space: nxest
or nyest too small, or s too small.
The weighted least-squares spline corresponds to the current set of
knots.
  warnings.warn(message)
/home/alk/miniconda3/envs/3d/lib/python3.9/site-packages/scipy/interpolate/_fitpack2.py:1419: UserWarning: 
The required storage space exceeds the available storage space: nxest
or nyest too small, or s too small.
The weighted least-squares spline corresponds to the current set of
knots.
  warnings.warn(message)
/home/alk/miniconda3/envs/3d/lib/python3.9/site-packages/scipy/interpolate/_fitpack2.py:1419: UserWarning: 
The required storage space exceeds the available storage space: nxest
or nyest too small, or s too small.
The weighted least-squares spline corresponds to the current set of
knots.
  warnings.warn(message)
/home/alk/miniconda3/envs/3d/lib/python3.9/site-packages/s

In [13]:
res_striped = run_test('striped')

  0%|          | 0/16 [00:00<?, ?it/s]

In [16]:
df_no_noise = pd.DataFrame(res_no_noise)
df_low_noise = pd.DataFrame(res_low_noise)
df_med_noise = pd.DataFrame(res_med_noise)
df_high_noise = pd.DataFrame(res_high_noise)
df_gradient = pd.DataFrame(res_gradient)
df_striped = pd.DataFrame(res_striped)
df_test_res = pd.concat((df_no_noise,
                         df_low_noise,
                         df_med_noise,
                         df_high_noise,
                         df_gradient,
                         df_striped))
df_test_res

,class,dataset,params,score
0,no_noise,galera100k,"{'gamma': 1, 'k': 30, 'kernel': 'gaussian', 's...",8.258374
1,no_noise,icosahedron100k,"{'gamma': 1, 'k': 30, 'kernel': 'gaussian', 's...",3.184624
2,no_noise,netsuke100k,"{'gamma': 1, 'k': 30, 'kernel': 'gaussian', 's...",6.518083
3,no_noise,Cup34100k,"{'gamma': 1, 'k': 30, 'kernel': 'gaussian', 's...",7.148996
4,no_noise,sphere100k,"{'gamma': 1, 'k': 30, 'kernel': 'gaussian', 's...",0.611236
...,...,...,...,...
11,striped,pipe100k_ddist_minmax_layers,"{'gamma': 1, 'k': 30, 'kernel': 'gaussian', 's...",4.741156
12,striped,pipe_curve100k_ddist_minmax_layers,"{'gamma': 1, 'k': 30, 'kernel': 'gaussian', 's...",4.650801
13,striped,column100k_ddist_minmax_layers,"{'gamma': 1, 'k': 30, 'kernel': 'gaussian', 's...",22.510935
14,striped,column_head100k_ddist_minmax_layers,"{'gamma': 1, 'k': 30, 'kernel': 'gaussian', 's...",22.522018


In [19]:
df_test_res_grouped = df_test_res[['class', 'score']].groupby('class').mean()
df_test_res_grouped

,score
class,
gradient,7.816128
high_noise,27.745462
low_noise,11.363698
med_noise,19.712657
no_noise,7.041440
striped,8.004623


In [21]:
df_test_res_grouped.mean().item()

13.614001050142008

In [22]:
save = False
fname = os.path.join('estimator', 'test_res.csv')

if save:
    df_test_res.to_csv(fname)